In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import numpy as np

: 

In [ ]:
# cluster_info_df = pd.DataFrame.from_records([pd.read_pickle("../data/closest_haplotypes_all_trimmed_2_fixed_3444_tn93")])
# records = SeqIO.parse("../data/sequences_2021-03-04_08-34_trimmed_50_l_r_no_amb_aligned_before_march_5_3757_total_sorted.fasta", 'fasta')
# records = tuple(SeqIO.parse("../data/cog_all_trimmed_no_amb_aligned.sorted.fasta", 'fasta'))
records = tuple(SeqIO.parse("../data/sequences_2021-03-04_08-34_trimmed_50_l_r_no_amb_aligned_before_march_5_3757_total_sorted.fasta", 'fasta'))

# with open("../data/closest_haplotypes_all_trimmed_2_fixed_3444_tn93","rb") as pkl:
#     data=pickle.load(pkl)
# cluster_info_dict = dict(pd.read_pickle("../data/closest_haplotypes_all_trimmed_2_fixed_3444_tn93"))
cluster_info_dict = dict(pd.read_pickle("../data/closest_haplotypes_500k_trimmed_tn93_before_march_5_0_005"))
# print(len(tuple(records)))
# print(data)

In [ ]:
sequence_name=[]
sequences=[]
for seq_record in records:  # (generator)
        sequence_name.append(seq_record.id)
        sequences.append(seq_record.seq)

In [ ]:
cluster=[]
for val in sequence_name:
    cluster.append(cluster_info_dict[val])

In [ ]:
fasta_dict = {'sequence_name':sequence_name , 'cluster_name':cluster,'sequence': sequences}
fasta_df = pd.DataFrame(data=fasta_dict).drop_duplicates(subset='sequence_name', keep="last")

In [ ]:
fasta_df

,sequence_name,cluster_name,sequence
0,Malaysia/IMR_WC1170/2020,4,"(A, A, G, A, A, A, G, G, T, T, T, A, T, A, C, ..."
1,Hangzhou/ZJU-06/2020,0,"(-, T, T, C, A, A, C, G, T, T, T, A, T, A, C, ..."
2,Hangzhou/ZJU-08/2020,3,"(-, -, -, -, G, A, G, A, T, T, T, A, T, A, C, ..."
3,Denmark/SSI-102/2020,2,"(-, -, -, -, A, A, G, C, T, T, T, A, T, A, C, ..."
4,Hangzhou/ZJU-03/2020,4,"(-, -, -, -, -, A, G, A, T, T, T, A, T, A, C, ..."
...,...,...,...
3683,England/20099071904/2020,13,"(-, -, -, -, -, -, -, -, -, -, -, -, -, -, -, ..."
3684,USA/UT-01320/2020,13,"(-, -, -, -, -, -, -, -, -, -, -, -, -, -, -, ..."
3685,USA/CA-SCCPHD-UC123/2020,13,"(-, -, -, -, -, -, -, -, -, -, -, -, -, -, -, ..."
3686,USA/CO-CDPHE-2003040178/2020,13,"(-, -, -, -, -, -, -, -, -, -, -, -, -, -, -, ..."


In [ ]:
seq_data=[]
for val in fasta_df.values:
    seq=list(val[2])
    seq.insert(0,val[0])
    seq_data.append(seq)


In [ ]:
seq_data = np.array(seq_data)

In [ ]:
seq_data

array([['Malaysia/IMR_WC1170/2020', 'A', 'A', ..., '-', '-', '-'],
       ['Hangzhou/ZJU-06/2020', '-', 'T', ..., '-', '-', '-'],
       ['Hangzhou/ZJU-08/2020', '-', '-', ..., '-', '-', '-'],
       ...,
       ['USA/CA-SCCPHD-UC123/2020', '-', '-', ..., '-', '-', '-'],
       ['USA/CO-CDPHE-2003040178/2020', '-', '-', ..., '-', '-', '-'],
       ['Australia/WA27/2020', '-', '-', ..., '-', '-', '-']],
      dtype='<U47')

In [ ]:
len(seq_data.T[1:])

29891

In [ ]:
from tqdm import tqdm
import math

In [ ]:
entropys_list=[]
for val in tqdm(seq_data.T[1:]):
    dash,A,C,T,G,entropy=0,0,0,0,0,0
    for neu in val:
        if neu == "-":
            dash+=1
        elif neu == "A":
            A+=1
        elif neu == "C":
            C+=1
        elif neu == "T":
            T+=1
        elif neu == "G":
            G+=1
    total = A+C+T+G
    if (total > 0):
        n_a_freq = A/ total
        n_c_freq = C/ total
        n_t_freq = T/ total
        n_g_freq = G/ total
        entropy_a = n_a_freq * math.log2(n_a_freq) if n_a_freq > 0 else 0
        entropy_c = n_c_freq * math.log2(n_c_freq) if n_c_freq > 0 else 0
        entropy_g = n_t_freq * math.log2(n_t_freq) if n_t_freq > 0 else 0
        entropy_t = n_g_freq * math.log2(n_g_freq) if n_g_freq > 0 else 0
        entropy = entropy + entropy_a + entropy_c + entropy_g + entropy_t
    entropys_list.append(abs(entropy))

    

100%|██████████| 29891/29891 [02:20<00:00, 213.09it/s]


In [ ]:
entropys_list
sorted_tags_wrt_entropy = np.argsort(entropys_list)

In [ ]:
sorted_tags_wrt_entropy[::-1][:100]

array([29890,     6,     7, 29888, 29887,     1,     2,     3,   240,
        3036, 23402, 14407, 29889,     4, 28143,  8781, 11082, 29867,
          30, 29866,    18, 28882, 28881, 28880,    20,    10, 26143,
       29869, 14804, 29863, 29861, 29860, 29858, 29857, 29855,    12,
          15,    16,    19, 29880,    25, 18059, 29872,    24, 29741,
       29871,    27,    26,    22, 17857,    56, 17746,    54, 27045,
       29868,    36,    34, 25562, 29856,    38,  1396, 29854, 29853,
       28687,   152,  9476,  1058, 28862, 28656,   154, 25978, 15323,
       25349,  5571,  2557,  2479, 26529,   513, 17246, 29094,  1439,
       29846,  2890, 17372, 29634, 20267, 29302, 28076, 11751, 28877,
       29708,  9961, 24033, 13928, 29710, 21706, 26728,  4401, 18876,
         265])

# Find N Tags

In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import numpy as np
import math

In [59]:
records = tuple(SeqIO.parse("../data/sequences_2021-03-04_08-34_trimmed_50_l_r_no_amb_aligned_before_march_5_3757_total_sorted.fasta", 'fasta'))
cluster_info_dict = dict(pd.read_pickle("../data/closest_haplotypes_500k_trimmed_tn93_before_march_5_0_005"))
sequence_name,sequences,cluster=[],[],[]

for seq_record in records:  # (generator)
        sequence_name.append(seq_record.id)
        sequences.append(seq_record.seq)
for val in sequence_name:
    cluster.append(cluster_info_dict[val])
fasta_dict = {'sequence_name':sequence_name , 'cluster_name':cluster,'sequence': sequences}
fasta_df = pd.DataFrame(data=fasta_dict).drop_duplicates(subset='sequence_name', keep="last")

In [60]:
# no_of_columns=1000
def find_n_tags(seq_data, n_tags_index):
    
    seq_names = []
    sequences = []
    for val in seq_data:
        inner=[]
        seq=""
        for idx in n_tags_index:
            inner.append(val[idx])
        sequences.append(Seq(seq.join(inner)))
        seq_names.append(val[0])
    return sequences, seq_names




In [ ]:
def sort_tags_based_on_entropy(seq_data):
    entropys_list=[]
    for val in seq_data.T[1:]:
        dash,A,C,T,G,entropy=0,0,0,0,0,0
        for neu in val:
            if neu == "-":
                dash+=1
            elif neu == "A":
                A+=1
            elif neu == "C":
                C+=1
            elif neu == "T":
                T+=1
            elif neu == "G":
                G+=1
        total = A+C+T+G
        if (total > 0):
            n_a_freq = A/ total
            n_c_freq = C/ total
            n_t_freq = T/ total
            n_g_freq = G/ total
            entropy_a = n_a_freq * math.log2(n_a_freq) if n_a_freq > 0 else 0
            entropy_c = n_c_freq * math.log2(n_c_freq) if n_c_freq > 0 else 0
            entropy_g = n_t_freq * math.log2(n_t_freq) if n_t_freq > 0 else 0
            entropy_t = n_g_freq * math.log2(n_g_freq) if n_g_freq > 0 else 0
            entropy = entropy + entropy_a + entropy_c + entropy_g + entropy_t
        entropys_list.append(abs(entropy))
    return entropys_list

In [61]:
def sort_tags_based_on_hamming(seq_data):
    dist_list=[]
    for val in seq_data.T[1:]:
        dash,A,C,T,G,hamming=0,0,0,0,0,0
        for neu in val:
            if neu == "-":
                dash+=1
            elif neu == "A":
                A+=1
            elif neu == "C":
                C+=1
            elif neu == "T":
                T+=1
            elif neu == "G":
                G+=1
        total = A + C + T + G + dash
        max_actg = max(A,C,T,G)
        hamming = total - (max_actg + dash)
        dist_list.append(hamming)
    return dist_list

In [62]:
def get_n_tags(no_of_columns):
    print(no_of_columns)
    seq_data=[]
    for val in fasta_df.values:
        seq=list(val[2])
        seq.insert(0,val[0])
        seq_data.append(seq)
        
    # for entropy
    # entropys_list = sort_tags_based_on_entropy(np.array(seq_data))
    # sorted_tags_wrt_entropy = np.argsort(entropys_list)
    # n_tags_index = [x + 1 for x in sorted_tags_wrt_entropy[::-1][:no_of_columns]]
    dist_list = sort_tags_based_on_hamming(np.array(seq_data))
    sorted_tags_wrt_hamming = np.argsort(dist_list)
    n_tags_index = [x + 1 for x in sorted_tags_wrt_hamming[::-1][:no_of_columns]]
    
    return find_n_tags(seq_data, n_tags_index)
    
    
    


In [63]:
seqs, seq_names = get_n_tags(1000)

1000


In [64]:
cluster=[]
for val in seq_names:
    cluster.append(cluster_info_dict[val])

In [65]:
fasta_dict = {'sequence_name':seq_names , 'cluster_name':cluster,'sequence': seqs}
fasta_df = pd.DataFrame(data=fasta_dict).drop_duplicates(subset='sequence_name', keep="last")

In [66]:
len(fasta_df["sequence"][0])

1000

In [67]:
np.save("../data/1k_tags_3k_hamm.pkl",fasta_df.values)